# Engineering Notebook: Design Exercise 4

### Initial thoughts
- How can we make the system replicated but also restartable?
- How do we know how many replicas are in the system?
- Maybe we can do a leader-replicas system where the leader is fixed at some port (e.g. 5000) and when none of the replicas can reach the port, they hold a leader election and the one with smallest ID also assumes port 5000
    - This would fail in the case of a network partition but I think that's fine
    - Also, I'm not sure how this would work with the connection we keep open for push messages, since in any case, once that replica dies, we need to establish a new connection. We might need to change that mechanism to one where the server opens a connection with the client (treating the client as a server) and sends a single message once, since this could come from any one of the replicas
    - Setting a specific port does not really work across machines; you'd need to be able to claim a (virtual) IP address which will be kind of hard for this exercise
- What if instead, we have a config with possible machines (e.g. 7 possible host/port combinations)? This way, when each machine joins, it can check all possible machines for a heartbeat, find a leader or conduct a leader election
    - Simpler approach: leader sends heartbeat to all replicas; the replicas don't need to know about the other replicas until it's time for an election or they become a leader
    - Client should be able to send their request to any replica, if the receiving replica is not the leader, it should forward it to the leader, otherwise the leader needs to notify the client of their identity
        - Side question: does this defeat the sense in which from the outside, replicas are supposed to be indistinguishable? I feel like yes but I also don't know how to do this without requiring the leader to have a specific IP

### Changes to our code from Exercise 1
- Let's make requests kind of like HTTP in that they always open a new socket connection. The initial log in/registration from the client should include a response address to which push notifications can be sent by the server

### Open Problems
- What happens if we're in the process of an election and a client sends different requests to two different replicas? How do we do ordering in this case?
- With the separate internal port, what happens if just the internal/external thread crashes?
- If a replica fails in between accepting an election request and its own response with election OK, the replica that called the election will be stuck in a loop. Should probably time out?
- If I create a login token on the leader replica, how do the other replicas learn about this?
    - OK we're getting rid of the login token and instead we send the username and password hash along with every authenticated request.

### Bringing replicas up to date
- Right now, replicas are brought up to date whenever there is a request and they say that they haven't received all of the prior requests. It might be nicer to have them automatically be brought up to date, this would increase fault tolerance
- Most things work nicely now, **except** that when a message is delivered immediately because the recipient is online, this is marked in the leader's database but not propagated to the other replicas (since it's not part of the job of the replica to deliver them directly)
    - OK update: kinda ugly but we're just having the client ask the server to mark the message as read, this way it's logged as an event and propagated

### Bug Reports
- There was a bug where the automatic election trigger timer did not restart if the replica previously called an election but was not elected the leader during that election. This is fixed now but is such a pain to test that I haven't written a test for it